In [ ]:
using PyBokeh

In [ ]:
ENV["SEEP_NO_GPU"]=false
using Seep, JLD
T = Float32
relu(x::ANode) = Seep.max!(0, x)
make_node(a::Array) = ANode(a)
make_node(x::Int...) = ANode(x...)
get_data(x) = x

include("data.jl")
data = IRData()
nothing

In [ ]:
@load "snapshot-brnn/statistics.jld"
f = line(figure(), collect(1:length(E)), E)
ylim(f, 0, 1)

In [ ]:
# Constants
const sequence_length = 50
const input_size = 2
const hidden_size = [10, 10]
const feature_size = 5
const output_size = 2
const batch_size = size(data.test, 3)

# Network
const X = ANode[make_node(input_size, batch_size) for i=1:sequence_length]
y = make_node(zeros(T, output_size, batch_size))

params = Flow[]
let 
    # Initialize nodes
    fe = Linear("e", input_size, hidden_size[1])
    fhf = Flow[]
    fhb = Flow[]
    for j = 1:length(hidden_size)
        k = j==1 ? 1 : j-1

        f = LSTM("h$(j)_f", hidden_size[k], hidden_size[j])
        push!(fhf, f)

        f = LSTM("h$(j)_b", hidden_size[k], hidden_size[j])
        push!(fhb, f)
    end
    fxf = Linear("xf", hidden_size[end], feature_size)
    fxb = Linear("xb", hidden_size[end], feature_size)
    ff = Linear("features", feature_size, feature_size)
    fy = Linear("output", feature_size, output_size)
    push!(params, fe, fhf..., fhb..., fxf, fxb, ff, fy)

    # Initialize LSTM variables
    hf = ANode[make_node(zeros(T, hidden_size[i], batch_size)) for i = 1:length(hidden_size)]
    cf = ANode[make_node(zeros(T, hidden_size[i], batch_size)) for i = 1:length(hidden_size)]
    hb = ANode[make_node(zeros(T, hidden_size[i], batch_size)) for i = 1:length(hidden_size)]
    cb = ANode[make_node(zeros(T, hidden_size[i], batch_size)) for i = 1:length(hidden_size)]

    # Build graph
    xf = make_node(zeros(T, input_size, batch_size))
    xb = make_node(zeros(T, input_size, batch_size))
    for i = 1:length(X)
        xf = relu(fe(X[i]))
        xb = relu(fe(X[length(X)-i+1]))

        for j = 1:length(hidden_size)
            cf[j], hf[j] = fhf[j](xf, cf[j], hf[j])
            xf = hf[j]

            cb[j], hb[j] = fhb[j](xb, cb[j], hb[j])
            xb = hb[j]
        end
    end

    xf = fxf(xf)
    xb = fxb(xb)
    x = tanh(ff(xf + xb))
    y = softmax(fy(x), 1)
end

# Gradient Calculation for SGD
BRNN = instance(y)
nothing

In [ ]:
# Load Latest Snapshot
n_epoch_start = 0

if isdir("snapshot-brnn")
    f = filter(x->contains(x, "snapshot-"), readdir("snapshot-brnn"))
    if length(f) > 0
        epochs = map(x->parse(Int, split(split(x, ".")[1], "-")[end]), f)
        ind = indmax(epochs)
        n_epoch_start = epochs[ind]
        @show n_epoch_start
        load_snapshot(joinpath("snapshot-brnn", f[ind]), y)
    end
end


In [ ]:
function eval_test()
    xx = data.test

    for i = 1:length(X)
        xi = BRNN[X[i]]
        xi[:, :] = xx[:, i, :]
    end

    BRNN()
    BRNN[y]
end

In [ ]:
ys = eval_test()
;

In [ ]:
hist(figure(), vec(ys[1, :]), line_color="black")